In [1]:
from simtk import unit
import openmm
from openmm import app
import numpy as np
from sys import stdout
from openmmtools import integrators

In [13]:
pdb = app.PDBFile('ala2_fromURL.pdb')
topology = pdb.getTopology()
positions = pdb.getPositions(asNumpy=True).value_in_unit(unit.nanometer)


ff = app.ForceField('amber99sbildn.xml',"amber96_obc.xml")
system = ff.createSystem(
    topology=topology, 
    removeCMMotion=True,
    nonbondedMethod=app.NoCutoff,
    constraints=app.HBonds, 
    rigidWater=True
    )

platform = openmm.Platform.getPlatform(2)


Uncomment this cell to save the system as an xml file

In [14]:
# xml = openmm.XmlSerializer.serialize(system)
# xml_file = open('ala2_newxml_system.txt','w')
# xml_file.write(xml)
# xml_file.close()

In [24]:
temperature = 300.0 * unit.kelvin
collision_rate = 1.0 / unit.picosecond
timestep = 4.0 * unit.femtosecond
reportInterval = 2500
steps = 2.5E+6
fname = 'test'
#time = (steps*timestep).value_in_unit(unit.nanosecond)
parametersdict = {'Collision rate':collision_rate,'Temperature':temperature,'Timestep':timestep,'Report Interval':reportInterval}
import pickle
f_p = open(f'parameters{fname}.pkl','wb')
pickle.dump(parametersdict,f_p)
f_p.close


<function BufferedWriter.close>

In [16]:
# parameters = f'{temperature}\nCollision rate:{collision_rate}\nTimestep: {timestep}\nReport every {reportInterval} steps'
# with open('parameterfile.txt', 'w') as f:
#     f.write(parameters)

In [17]:

from mdtraj import reporters

integrator = integrators.LangevinIntegrator(temperature=temperature,collision_rate=collision_rate,timestep=timestep)
#integrator.setConstraintTolerance(0.00001)
#integrator = openmm.VerletIntegrator(timestep)
properties_dict = {}
properties_dict["DeviceIndex"] = "2"
simulation = app.Simulation(topology, system, integrator,platform,platformProperties=properties_dict)
#print(platform.getPropertyValue(simulation.context,property='Precision'))
#print(simulation.context.getPlatform())
simulation.context.setPositions(positions)
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)

simulation.reporters.append(app.StateDataReporter(stdout, reportInterval=reportInterval, step=True, potentialEnergy=True,temperature=True))
simulation.reporters.append(app.DCDReporter(f'{fname}.dcd',reportInterval))
#h5_reporter = reporters.HDF5Reporter('output.h5',reportInterval)
#simulation.reporters.append(h5_reporter)


In [18]:
simulation.step(steps)
#h5_reporter.close()

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
2500,-74.95901489257812,266.85421578200936
5000,-74.1497802734375,256.81824124074507
7500,-67.72030639648438,279.42835840779264
10000,-71.10049438476562,234.18163946732898
12500,-80.30001831054688,292.1079536680365
15000,-80.93338012695312,353.8335494244482
17500,-85.78375244140625,252.81201271326702
20000,-83.43826293945312,179.31008711340527
22500,-77.3941650390625,251.4482893846634
25000,-48.411712646484375,180.53116780662174
27500,-85.15420532226562,222.38076818156455
30000,-58.0726318359375,278.70816423406944
32500,-74.3123779296875,243.73126523498703
35000,-70.92758178710938,345.2579346389652
37500,-57.165313720703125,219.17632872582692
40000,-41.499114990234375,279.1368478151409
42500,-91.140380859375,259.6011220502767
45000,-71.2930908203125,251.46577786115137
47500,-85.19873046875,272.1033316215345
50000,-64.78070068359375,302.206991079764
52500,-70.23397827148438,241.19469846158685
55000,-75.64678955078125,230.67748494848